# Get model package from the model registry

In [2]:
%cd ..

/root/mll-mlops-model-pipeline-template


In [3]:
import argparse
import boto3
import logging
import os
from botocore.exceptions import ClientError
import tarfile
import zipfile
import sagemaker
import yaml

logger = logging.getLogger(__name__)
sm_client = boto3.client("sagemaker")

# LOAD CONFIG FILE (DON'T FORGET TO MODIFY TO BE ABLE TO USE THE RESPECTIVE VERTION)
with open("cfg/model_build.yaml") as f: config = yaml.load(f, Loader=yaml.SafeLoader)

# IAM ROLE
iam_role = sagemaker.get_execution_role()

# Sagemaker session
sagemaker_session = sagemaker.session.Session()


def get_approved_package(model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Fetch more packages if none returned with continuation token
        while len(approved_packages) == 0 and "NextToken" in response:
            logger.debug("Getting more packages for token: {}".format(response["NextToken"]))
            response = sm_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = (
                f"No approved ModelPackage found for ModelPackageGroup: {model_package_group_name}"
            )
            logger.error(error_message)
            raise Exception(error_message)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        logger.info(f"Identified the latest approved model package: {model_package_arn}")
        return approved_packages[0]
        # return model_package_arn
    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        logger.error(error_message)
        raise Exception(error_message)

Get model package  metadata from model registry

In [18]:
sm_client = boto3.client("sagemaker")
model_package_group_name = config['model_registry']['model_package_group_name']+"-"+"development"
pck = get_approved_package(
    model_package_group_name
)
model_description = sm_client.describe_model_package(ModelPackageName=pck["ModelPackageArn"])

model_description

{'ModelPackageGroupName': 'mll-mlops-model-template-development',
 'ModelPackageVersion': 13,
 'ModelPackageArn': 'arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-mlops-model-template-development/13',
 'CreationTime': datetime.datetime(2022, 12, 27, 9, 8, 41, 343000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
    'ImageDigest': 'sha256:77dbe273dea0c6430bee29eb31b5a8ebf90c756f24d130070473aa7f7164ab1f',
    'ModelDataUrl': 's3://mll-tfm-v3-model-pipeline-development-961105418118/PrepareData-49936e2ab4029a1bf4f7e73317fbfead/output/model/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'preprocessing_inference.py',
     'SAGEMAKER_REGION': 'eu-central-1',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://mll-tfm-v3-model-pipeline-development-961105418118/sagemaker-scikit-learn-2022-12-27-09-01-14-791/sourcedir.tar.gz'}

# Deploy registered model endpoint

In [20]:
from sagemaker import ModelPackage
import time

# Fetch model endpoint config
model_package_arn = model_description["ModelPackageArn"]
model = ModelPackage(
    role=iam_role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Generate endpoint name
endpoint_name = "DEMO-mll-tfm-v3-endpoint-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("EndpointName= {}".format(endpoint_name))

# Deploy endpoint
model.deploy(initial_instance_count=1, 
             instance_type=config['model_registry']['instance_type'], 
             endpoint_name=endpoint_name)

EndpointName= DEMO-mll-tfm-v3-endpoint-2022-12-27-09-09-54
------!

# Build endpoint object

In [21]:
from sagemaker.predictor import Predictor
#endpoint_name = "mlops-template-staging"
predictor = Predictor(endpoint_name=endpoint_name)

# Test endpoint

In [22]:
import json

# input example provided to tfm
content_type = "application/json"
request_body = {
      "product": "TFM",
      "channel": "WEB",
      "sourceMarket": "",
      "brand": "",
      "productType": "",
      "outDepartureAirport": "FRA",
      "outArrivalAirport": "JFK",
      "outDate": "2023-04-10T14:15:00",
      "outTime": "2023-04-10T14:15:00",
      "outCarrier": "WA | KL | DL | ",
      "outFlightNumber": "1768 | 0643 | 6106 | ",
      "outDuration": 600,
      "outSegments": 2,
      "inDepartureAirport": "JFK",
      "inArrivalAirport": "FRA",
      "inDate": "2023-04-16T19:40:00",
      "inTime": "2023-04-16T19:40:00",
      "inCarrier": "WA | KL | DL | ",
      "inFlightNumber": "1768 | 0643 | 6106 | ",
      "inDuration": 490,
      "inSegments": 1,
      "journeyType": "ROUNDTRIP",
      "tariffType": "PU",
      "fareBase": "",
      "corporateId": "",
      "baggage": "",
      "currency": "EUR",
      "vatPercentage": 0,
      "supplierPriceAdult": 610.17,
      "supplierPriceChild": 0,
      "supplierPriceInfant": 0,
      "adult": 1,
      "child": 0,
      "infant": 0,
      "source": "KL",
      "method": "MARGINBRAIN",
      "sessionId": "3a3874f1-927a-4ca7-b5a4-bd46f92fce5b",
      "requestId": "",
      "vcc": "KL",
      "event_type": "searchFlight"
    }

data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

predict_payload = predictor.predict(payload, initial_args={"ContentType": content_type})
print(predict_payload.decode("utf-8"))

{"product": "TFM", "channel": "WEB", "sourceMarket": "", "brand": "", "productType": "", "outDepartureAirport": "FRA", "outArrivalAirport": "JFK", "outDate": "2023-04-10T14:15:00", "outTime": "2023-04-10T14:15:00", "outCarrier": "WA | KL | DL | ", "outFlightNumber": "1768 | 0643 | 6106 | ", "outDuration": 600, "outSegments": 2, "inDepartureAirport": "JFK", "inArrivalAirport": "FRA", "inDate": "2023-04-16T19:40:00", "inTime": "2023-04-16T19:40:00", "inCarrier": "WA | KL | DL | ", "inFlightNumber": "1768 | 0643 | 6106 | ", "inDuration": 490, "inSegments": 1, "journeyType": "ROUNDTRIP", "tariffType": "PU", "fareBase": "", "corporateId": "", "baggage": "", "currency": "EUR", "vatPercentage": 0, "supplierPriceAdult": 610.17, "supplierPriceChild": 0, "supplierPriceInfant": 0, "adult": 1, "child": 0, "infant": 0, "source": "KL", "method": "MARGINBRAIN", "sessionId": "3a3874f1-927a-4ca7-b5a4-bd46f92fce5b", "requestId": "", "vcc": "KL", "event_type": "searchFlight", "inference_data": [[-0.14188

# Clean-up
Delete the resources created for this example to avoid any unintended charges

In [54]:
sm_client = boto3.client("sagemaker")

for d in sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)[
    "ModelPackageSummaryList"
]:
    print(d["ModelPackageArn"])
    sm_client.delete_model_package(ModelPackageName=d["ModelPackageArn"])

sm_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/19
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/18
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/17
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/16
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/15
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/14
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/13
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/12
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/11
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/10
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/9
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/8
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v3/7
arn:aws:sagemaker:eu-central-1:961105418118:model-package/mll-tfm-v

{'ResponseMetadata': {'RequestId': '15793474-8e06-4fe7-b680-e168a3e6fe10',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '15793474-8e06-4fe7-b680-e168a3e6fe10',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 15 Nov 2022 13:53:46 GMT'},
  'RetryAttempts': 0}}

In [24]:
predictor.delete_endpoint()